In [1]:
import islpy as isl
from src.io_tools import *
from src.graph_tools import *
from src.swap_tools import *

In [2]:
json_file_path = 'benchmarks/polyhedral/cases/bigd.json'

data = json_file_to_isl(json_file_path)

In [3]:
domain = data["domain"]
access = data["read_dependencies"]
schedule = data["schedule"]
qops = data["Qops"]

In [4]:
physical_qubits_domain = isl.Set(f"{{ Q[i] : 1 <= i <= {qops} }}")

In [5]:
# backend

graph = defaultdict(list)
    
graph['Q[1]'] = ['Q[2]', 'Q[6]']
graph['Q[2]'] = ['Q[1]', 'Q[3]', 'Q[7]']
graph['Q[3]'] = ['Q[2]', 'Q[4]', 'Q[8]']
graph['Q[4]'] = ['Q[3]', 'Q[5]', 'Q[9]']
graph['Q[5]'] = ['Q[4]', 'Q[10]']
graph['Q[6]'] = ['Q[1]', 'Q[7]', 'Q[11]']
graph['Q[7]'] = ['Q[2]', 'Q[6]', 'Q[8]', 'Q[12]']
graph['Q[8]'] = ['Q[3]', 'Q[7]', 'Q[9]', 'Q[13]']
graph['Q[9]'] = ['Q[4]', 'Q[8]', 'Q[10]', 'Q[14]']
graph['Q[10]'] = ['Q[5]', 'Q[9]', 'Q[15]']
graph['Q[11]'] = ['Q[6]', 'Q[12]', 'Q[16]']
graph['Q[12]'] = ['Q[7]', 'Q[11]', 'Q[13]', 'Q[17]']
graph['Q[13]'] = ['Q[8]', 'Q[12]', 'Q[14]', 'Q[18]']
graph['Q[14]'] = ['Q[9]', 'Q[13]', 'Q[15]', 'Q[19]']
graph['Q[15]'] = ['Q[10]', 'Q[14]', 'Q[20]']
graph['Q[16]'] = ['Q[11]', 'Q[17]']
graph['Q[17]'] = ['Q[12]', 'Q[16]', 'Q[18]']
graph['Q[18]'] = ['Q[13]', 'Q[17]', 'Q[19]']
graph['Q[19]'] = ['Q[14]', 'Q[18]', 'Q[20]']
graph['Q[20]'] = ['Q[15]', 'Q[19]']


backend_disconnected_edges =  extract_edges_map(graph)
shortest_paths = extract_shortest_paths(graph)

In [6]:
initial_mapping = isl.Map(f"{{ q[i] -> Q[i] : 1<=i<={qops} }}")

In [7]:
programme = schedule.reverse().apply_range(access)

In [8]:
mapping = initial_mapping
swap_count = 0

In [9]:
while not programme.is_empty():
    programme_mapped = programme.apply_range(mapping)
    programme_access = programme_mapped.flat_range_product(programme_mapped).intersect_range(isl.Set("{ [i, j] : i > j }"))
    first_disconnected_edge = programme_access.apply_range(backend_disconnected_edges).domain().lexmin()
    if first_disconnected_edge.is_empty():
        break

    disconected_qubits = programme_mapped.intersect_domain(first_disconnected_edge).range().as_set()
    first_qubit , second_qubit = disconected_qubits.lexmin().to_str().replace("{", "").replace("}", "").strip() , disconected_qubits.lexmax().to_str().replace("{", "").replace("}", "").strip()
    swap_count += shortest_paths[first_qubit]['costs'][second_qubit] -1
    new_domain = programme.domain().as_set().lex_ge_set(first_disconnected_edge.as_set()).domain()
    programme = programme.intersect_domain(new_domain)
    swap_map = swaps_to_isl_map(shortest_paths[first_qubit]['paths'][second_qubit])
    mapping = apply_swaps_to_logical_qubits_map(swap_map,mapping,physical_qubits_domain)


In [9]:
swap_count

0